In [9]:
import re
import json
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import nltk
from gensim.utils import simple_preprocess

In [10]:
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'] # Hier die Jahre angeben, die geladen werden sollen (final: 2016-2023)
path = "/Users/jan/Documents/Promotion/BIBB GW/hdd/Pflege/"#Hier der Pfad zu den JSON-Dateien!
with open(f"{path}Pflege_2016.json", 'r') as file:
    output = json.load(file)
docs = [str(value) for value in output["conditions_description"].values()]
del output

In [11]:
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.
# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]
# Remove stop words
stop_words = set(stopwords.words('german'))
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]
docs = remove_stopwords(docs)

In [12]:
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [13]:
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [14]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [15]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [16]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -1.8664.
[([(0.024328569, 'stellenangebots'),
   (0.024307186, 'karte'),
   (0.02147389, 'tarifvertrag'),
   (0.018760942, 'teilzeit'),
   (0.016613042, 'stellen'),
   (0.01575984, 'vergütung'),
   (0.014896133, 'zusatzleistungen'),
   (0.0144716855, 'findet'),
   (0.014430503, 'anwendung'),
   (0.014427432, 'besetzung'),
   (0.014421018, 'arbeitsverhältnis'),
   (0.01434932, 'stellenangebotes'),
   (0.014345701, 'besetzung_stellenangebotes'),
   (0.014142353, 'arbeitszeit'),
   (0.013601407, 'tätigkeit'),
   (0.01318517, 'befristung'),
   (0.013135544, 'vollzeit'),
   (0.012795153, 'finden'),
   (0.012690846, 'beginn'),
   (0.012687556, 'informationen')],
  -0.3333707825924487),
 ([(0.020232785, 'igz'),
   (0.015975852, 'tarifvertrag'),
   (0.015240662, 'vergütung'),
   (0.012396032, 'stets'),
   (0.012326698, 'monat'),
   (0.0122197755, 'std'),
   (0.012126703, 'urlaubs'),
   (0.011872186, 'urlaubs_weihnachtsgeld'),
   (0.011867614, 'weihnachtsgeld'),
   (0.0